In [ ]:
# For running in Colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U xformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U einops
!pip install -q -U auto-gptq
!pip install -q -U optimum
!pip install -q -U nvidia-ml-py3
!pip install -q -U pynvml

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from pynvml import *
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model
import time, torch
import matplotlib.pyplot as plt

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [ ]:
# Model ID
base_model_id = "microsoft/phi-1_5"

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id  , use_fast=True, max_length=250)

# Quantize
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
          base_model_id, trust_remote_code=True, quantization_config=bnb_config, device_map={"": 0}
)

# The code was run in Google Colab - check GPU usage
print(print_gpu_utilization())

In [ ]:
# PREPROCESSING OF DATA - done once, and henceforth the data is loaded from storage.

# dataset = load_dataset("tatsu-lab/alpaca")
# dataset = dataset["train"].train_test_split(test_size=0.2)

# dataset_train = dataset["train"]
# dataset_test = dataset["test"]

# # Define a filter function
# def filter_empty_inputs(example):
#     return example['input'] != ""

# # Apply the filter function to the dataset
# dataset_train = dataset_train.filter(filter_empty_inputs)
# dataset_test = dataset_test.filter(filter_empty_inputs)

# dataset_train
# dataset_test

# ##################################################

# dataset_train.to_json("/content/drive/MyDrive/Colab Notebooks/data/alpaca_dataset/dataset_train.json")
# dataset_test.to_json("/content/drive/MyDrive/Colab Notebooks/data/alpaca_dataset/dataset_test.json")

In [ ]:
# Load data
dataset_train = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/data/alpaca_dataset/dataset_train.json")
dataset_test = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/data/alpaca_dataset/dataset_test.json")

# Format
dataset_train = dataset_train["train"]
dataset_test = dataset_test["train"]

# Padding
tokenizer.padding_side = 'left'
tokenizer.pad_token = tokenizer.unk_token

# PEFT config
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ["Wqkv", "out_proj"]
)

# Tokenizer function
def tok(sample):
    model_inps = tokenizer(sample["text"], padding=True, max_length=500, truncation=True)
    return model_inps

# Tokenize data
tokenized_training_data = dataset_train.map(tok, batched=True)
tokenized_test_data = dataset_test.map(tok, batched=True)

In [ ]:
# Add PEFT config to model specs
model = get_peft_model(model, peft_config)

# Gradient checkpointing
model.gradient_checkpointing=True

# Training arguments
training_arguments = TrainingArguments(
        output_dir="/content/drive/MyDrive/Colab Notebooks/alpaca_peft/checkpoints",

        num_train_epochs=20,
        # Evaluating and saving by steps
        do_eval=True,
        logging_strategy="steps",
        logging_steps=200,
        evaluation_strategy="steps",
        eval_steps=200,
        save_strategy='steps',
        save_steps=200,

        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        per_device_eval_batch_size=4,

        learning_rate=4e-4,
        warmup_steps=100,
        lr_scheduler_type="cosine",
        remove_unused_columns=True
)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset = tokenized_training_data,
    eval_dataset = tokenized_test_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [ ]:
# TRAIN THE MODEL
trainer.train() #resume_from_checkpoint=True if resuming from previous training

In [ ]:
######## LOSS CURVES ########

# Extract loss values and steps
training_loss = [log['loss'] for log in trainer.state.log_history if 'loss' in log]
eval_loss = [log['eval_loss'] for log in trainer.state.log_history if 'eval_loss' in log]
steps = [log['step'] for log in trainer.state.log_history if 'loss' in log]
epochs = [log['epoch'] for log in trainer.state.log_history if 'loss' in log]

# Per epoch

# Plotting
plt.figure(figsize=(12, 6))

# Plot training loss
plt.plot(epochs, training_loss, label='Training Loss')
eval_epochs = [log['epoch'] for log in trainer.state.log_history if 'eval_loss' in log]
plt.plot(eval_epochs, eval_loss, label='Evaluation Loss', color='red')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Evaluation Loss Curves: PEFT of the phi-1_5 model on the Alpaca data subset')
plt.legend()

plt.savefig("/content/drive/MyDrive/Colab Notebooks/alpaca_peft/loss_epochs.png")

plt.show()

# Per step

# Plotting
plt.figure(figsize=(12, 6))

# Since evaluation might occur less frequently, its steps need to be calculated separately
plt.plot(steps, training_loss, label='Training Loss')
eval_steps = [log['step'] for log in trainer.state.log_history if 'eval_loss' in log]
plt.plot(eval_steps, eval_loss, label='Evaluation Loss', color='red')

plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Evaluation Loss Curves: PEFT of the phi-1_5 model on the Alpaca data subset')
plt.legend()

plt.savefig("/content/drive/MyDrive/Colab Notebooks/alpaca_peft/loss_steps.png")

plt.show()


In [ ]:
# Add the adapter to the model
base_model_id = "microsoft/phi-1_5"

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True, max_length=250)

# Quantization
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
          base_model_id, trust_remote_code=True, quantization_config=bnb_config, device_map={"": 0}
)

################## SPECIFY THE FINE TUNE CHECKPOINT ######################
adapter = "/content/drive/MyDrive/Colab Notebooks/alpaca_peft/checkpoints/checkpoint-1000"

# Add the adapter to the model
model = PeftModel.from_pretrained(model, adapter)

In [ ]:
##### HuggingFace credentials #####
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
##### PUSH TO HF HUB #####
model.push_to_hub("alekswael/phipaca_instruction", safe_serialization=True)
tokenizer.push_to_hub("alekswael/phipaca_instruction")